In [66]:
import os
from openai import OpenAI


In [67]:
from azure.ai.inference import ChatCompletionsClient
from azure.core.credentials import AzureKeyCredential

from azure.identity import DefaultAzureCredential, get_bearer_token_provider, AzureCliCredential



In [68]:
models = {
    "DeepSeek-R1": "DeepSeek-R1",
    "Phi-4-mini-instruct": "Phi-4-mini-instruct",
    "gpt-4o": "gpt-4o",
    "o3-mini": "o3-mini"
}

In [69]:
endpoint = os.getenv("AZURE_INFERENCE_SDK_ENDPOINT")

In [70]:
client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(os.getenv("AZURE_INFERENCE_SDK_ENDPOINT_KEY")),
    credential_scopes=["https://cognitiveservices.azure.com/.default"],

    )

In [71]:
from azure.ai.inference.models import (
    SystemMessage,
    UserMessage
)

In [72]:
def parse_think_tags(content):
    """
    Parse the <think> </think> tags out of the response in content
    """
    import re
    # Find all <think>...</think> tags and their content
    think_tags = re.findall(r"<think>(.*?)</think>", content)
    # match = re.search(r"<think>(.*?)</think>", content)
    # think_tags = match.group(0) if match else ""

    # Remove the <think>...</think> tags from the content
    content_without_think_tags = re.sub(r"<think>.*?</think>", "", content)

    return think_tags, content_without_think_tags

In [73]:
messages = [
    UserMessage(content="1 shirt needs 1 hour to dry, how long does it take to dry 3 shirts?"),
]

# DeepSeek-R1

In [74]:
response = client.complete(
    messages = messages,
    model = models['DeepSeek-R1']
)



In [75]:
print(response.choices[0].message.content)

<think>
Okay, so the problem is: "1 shirt needs 1 hour to dry. How long does it take to dry 3 shirts?" Hmm, at first glance, this seems straightforward, but maybe there's a trick here. Let me think through it step by step.

First, the basic idea. If one shirt takes 1 hour to dry when it's by itself, what happens when there are three shirts? Does drying time increase with more shirts, or does it stay the same? Well, if they're all drying simultaneously, like hanging next to each other on a clothesline or in a dryer, they might all dry at the same time. So maybe the answer is still 1 hour because they can all dry together. But wait, maybe there's a condition where drying multiple shirts could interfere with each other. Let me consider different scenarios.

Scenario 1: Drying outdoors on a line. If you hang three shirts next to each other, they all get exposed to the same sunlight and air. Unless they're so close that they block each other's airflow, they should dry in the same amount of 

# gpt-4o

In [76]:
# gpt-4o
response = client.complete(
    messages = messages,
    model = models["gpt-4o"]
)
print(response.choices[0].message.content)

It takes **1 hour** to dry 3 shirts, assuming all the shirts are drying at the same time under the same conditions (e.g., hung up together). The drying time is not additive; it depends on the drying process, which treats all shirts simultaneously rather than sequentially.


# Phi-4-mini-instruct

In [77]:
response = client.complete(
    messages = messages,
    model = models['Phi-4-mini-instruct']
)
print(response.choices[0].message.content)

If 1 shirt needs 1 hour to dry, then 3 shirts would also take 1 hour to dry, assuming they are placed close enough together to share the same drying space and the drying conditions (like airflow) remain constant. However, if they are separated and require individual space to dry, then it would take 3 hours for 3 shirts to dry.


# o3-mini

In [78]:
from openai import AzureOpenAI

In [79]:
aoai_client = AzureOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2024-12-01-preview")


In [80]:
chat_prompt = chat_prompt = [
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": messages[0].content
            }
        ]
    }]

In [81]:
completion = aoai_client.chat.completions.create(
    model="o3-mini",
    messages=messages,
    max_completion_tokens=100000,
    stop=None,
    stream=False ,

    reasoning_effort="high"  )

In [84]:
print(completion.choices[0].message.content)

If the shirts are dried at the same time (for example, hung on a clothesline or placed in a dryer that can handle multiple shirts), then it takes 1 hour to dry all three shirts, because they dry simultaneously rather than one after the other.


In [82]:
print(completion.to_json())

{
  "id": "chatcmpl-BBcaOeIpUQaAKHdVsSXKHYS3UiN5a",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "If the shirts are dried at the same time (for example, hung on a clothesline or placed in a dryer that can handle multiple shirts), then it takes 1 hour to dry all three shirts, because they dry simultaneously rather than one after the other.",
        "refusal": null,
        "role": "assistant"
      },
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "self_harm": {
          "filtered": false,
          "severity": "safe"
        },
        "sexual": {
          "filtered": false,
          "severity": "safe"
        },
        "violence": {
          "filtered": false,
          "severity": "safe"
        }
      }
    }
  ],
  "created": 1742109632,
  "model": "o3-mini-2025-01-31",
  "object": "chat.completion",
  "syste